In [ ]:
import os

import pandas as pd
from glom import T, glom
from pyprojroot import here

from municipal_performance_scraping.extractors import extract_performance_data

os.chdir(here())

In [172]:
target = extract_performance_data(1313, 2018)

In [175]:
def clean_string(s: str) -> str:
    return str(s).replace("\r\n", " ").strip()


def transform_raw_performance_response(raw_response: dict) -> pd.DataFrame:
    normalized_data = []

    domain_spec = {
        "name_ar": ("ar", clean_string),
        "name_fr": ("fr", clean_string),
        "max_score": ("maxdom", int),
        "score": ("notedom", int),
    }

    subdomain_spec = {
        "name_ar": ("nom.ar", clean_string),
        "name_fr": ("nom.fr", clean_string),
        "max_score": ("maxnote", int),
        "score": ("note", int),
    }

    criterion_spec = {
        "name_ar": ("nom.ar", clean_string),
        "name_fr": ("nom.fr", clean_string),
        "max_score": ("crmaxnote", int),
        "score": ("notecrit", int),
    }

    for domain_id, domain in raw_response.items():
        normalized_domain = {"parent_criterion_id": None, "criterion_id": domain_id}
        glommed = glom(domain, domain_spec)

        normalized_domain.update(glommed)
        normalized_data.append(normalized_domain)

        for subdomain_id, subdomain in glom(domain, ("sd", T.items())):
            normalized_subdomain = {
                "parent_criterion_id": domain_id,
                "criterion_id": f"{domain_id}{subdomain_id}",
            }
            glommed = glom(subdomain, subdomain_spec)

            normalized_subdomain.update(glommed)
            normalized_data.append(normalized_subdomain)

            for criterion_id, criterion in glom(subdomain, ("cr", T.items())):
                normalized_criterion = {
                    "parent_criterion_id": subdomain_id,
                    "criterion_id": f"{domain_id}{subdomain_id}{criterion_id}",
                }
                glommed = glom(criterion, criterion_spec)

                normalized_criterion.update(glommed)
                normalized_data.append(normalized_criterion)
    return pd.DataFrame(normalized_data)

In [176]:
transform_raw_performance_response(target)

,parent_criterion_id,criterion_id,name_ar,name_fr,max_score,score
0,None,10,الحوكمة,Gouvernance,30,21
1,10,101,تطبيق المقاربة التشاركية,Approche Participative,12,8
2,1,10114,انعقاد الجلسات التمهيدية لمجلس الجماعة المحلية,Quatre réunions ordinaires du conseil municipa...,4,0
3,1,10115,انعقاد الدورات العادية لمجلس الجماعة المحلية,Réunions préparatoires aux réunions du conseil...,4,4
4,1,10116,سعي الجماعة المحلية لاتخاذ القرارات (إستشارة ا...,Actions pour promouvoir une démarche participa...,4,4
5,10,102,الشفافية والنفاذ إلى المعلومة,Transparence et acces a l'information,8,8
6,2,10292,إطلاع المواطنين على الوثائق الأساسية للتصرف ال...,Les documents clefs de la commune sont rendus ...,4,4
7,2,10293,إعتماد الجماعة المحلية على و سائل إضافية لإعلا...,La commune utilise des moyens divers pour info...,4,4
8,10,103,آليات التصرف في الشكاوى,Mécanisme de gestion des plaintes,10,5
9,3,10395,تكليف إطار للتصرف في الشكاوى و معالجتها و متاب...,Point focal pour la gestion des plaintes nommé...,2,2
